In [1]:
import numpy as np, pandas as pd
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import *
import cv2 as cv
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('test_labels.csv')
# only keep finding and file name
df = df[['finding', 'filename']]
df = df[df.finding == 'Pneumonia/Viral/COVID-19'] # only keep covid images
df['filename'] = df['filename'].apply(lambda x: 'test_images/' + x)
df = df.dropna()
df.head()

,finding,filename
0,Pneumonia/Viral/COVID-19,test_images/auntminnie-a-2020_01_28_23_51_6665...
1,Pneumonia/Viral/COVID-19,test_images/auntminnie-b-2020_01_28_23_51_6665...
2,Pneumonia/Viral/COVID-19,test_images/auntminnie-c-2020_01_28_23_51_6665...
3,Pneumonia/Viral/COVID-19,test_images/auntminnie-d-2020_01_28_23_51_6665...
4,Pneumonia/Viral/COVID-19,test_images/nejmc2001573_f1a.jpeg


In [9]:
pd.read_csv('test_labels.csv').finding.value_counts()

Pneumonia/Viral/COVID-19                   584
todo                                        83
Pneumonia                                   81
Pneumonia/Fungal/Pneumocystis               30
No Finding                                  22
Pneumonia/Bacterial/Streptococcus           22
Tuberculosis                                18
Pneumonia/Viral/SARS                        16
Pneumonia/Lipoid                            13
Pneumonia/Bacterial/Mycoplasma              11
Pneumonia/Viral/MERS-CoV                    10
Pneumonia/Bacterial/Klebsiella              10
Pneumonia/Bacterial/Legionella              10
Pneumonia/Bacterial/Nocardia                 8
Pneumonia/Viral/Varicella                    6
Pneumonia/Viral/Influenza                    5
Pneumonia/Bacterial/E.Coli                   4
Pneumonia/Bacterial                          4
Pneumonia/Bacterial/Chlamydophila            3
Pneumonia/Viral/Herpes                       3
Pneumonia/Viral/Influenza/H1N1               2
Pneumonia/Fun

In [3]:
# Create dataset
class TestData(Dataset):
    def __init__(self, imgs, T):
        super().__init__()
        self.imgs = imgs
        self.transforms = T
    
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        path = self.imgs[idx]
        img = cv.cvtColor(cv.imread(path), cv.COLOR_BGR2RGB) / 255.
        img = self.transforms(img)
        return img.to('cuda')

In [4]:
T = Compose([
    ToTensor(),
    Resize((224,224))
])

# Drop bad paths from the dataset
bad_paths = []
for path in df.filename.values:
    r = cv.imread(path)
    if r is None:
        bad_paths.append(path)
        
df = df.drop(df.index[df['filename'].isin(bad_paths)])
paths = df.filename.values
test_ds = TestData(paths, T)
test_loader = DataLoader(test_ds, batch_size=10)
model = torch.load('mNet_v2')
criterion = torch.nn.CrossEntropyLoss()

In [5]:
def evaluate(model, dataloader, criterion):
    model.eval()
    P = torch.Tensor()
    
    for i, (imgs) in enumerate(dataloader):
        imgs = imgs.float().to('cuda')
        
        with torch.no_grad():
            output = model(imgs)        
        
        _, preds = torch.max(output, 1)
        P = torch.cat([P.to('cuda'), preds], dim=0)
    
    
    return P.cpu().numpy()

In [6]:
preds = evaluate(model, test_loader, criterion)

In [7]:
print(classification_report(np.zeros(len(preds)), preds))

              precision    recall  f1-score   support

         0.0       1.00      0.02      0.04       563
         1.0       0.00      0.00      0.00         0
         3.0       0.00      0.00      0.00         0

    accuracy                           0.02       563
   macro avg       0.33      0.01      0.01       563
weighted avg       1.00      0.02      0.04       563



/home/walt/learn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/walt/learn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/walt/learn/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
